In [1]:
import pandas as pd

In [35]:
leca_nog_file = '../ppi_ml/annotations/lists/leca_nogs.txt'
leca_nogs = [line.strip() for line in open(leca_nog_file, 'r')]
print(f'total # of LECA OGs: {len(leca_nogs)}')

total # of LECA OGs: 10092


# while I'm here... let's map these leca NOGs to categories

In [24]:
def fmt_ogs(x):
    if not x.startswith('COG'):
        if not x.startswith('KOG'):
            x = 'ENOG50'+x
    return x

In [17]:
cat_file = '../dollo_qfo/annotations/eggnog/cog_categories.csv'
annot_file = '../dollo_qfo/annotations/eggnog/e5.2759.og_annotations.csv'

In [66]:
cols = ['category', 'category_description']
cats = pd.read_csv(cat_file, header=None, names=cols)
cats.loc[len(cats.index)] = ['V', 'Defense mechanism']
cats

,category,category_description
0,A,RNA processing and modification
1,B,Chromatin Structure and dynamics
2,C,Energy production and conversion
3,D,Cell cycle control and mitosis
4,E,Amino Acid metabolis and transport
5,F,Nucleotide metabolism and transport
6,G,Carbohydrate metabolism and transport
7,H,Coenzyme metabolis
8,I,Lipid metabolism
9,J,Tranlsation


In [75]:
annots = pd.read_csv(annot_file)
annots['euNOG'] = [fmt_ogs(i) for i in annots['og']]
annots['category'] = annots['category'].str.replace('R','S')
annots['category'] = annots['category'].str.split(pat='')
annots = annots.explode('category').drop_duplicates().reset_index(drop=True)
annots = annots.merge(cats, how='left', left_on='category', right_on='category')
annots = annots.dropna(subset='category_description')
annots

,level,og,category,annotation,euNOG,category_description
1,2759,2TDXG,S,NaN,ENOG502TDXG,Function Unknown
3,2759,2QWWR,S,NaN,ENOG502QWWR,Function Unknown
5,2759,2SSDA,S,NaN,ENOG502SSDA,Function Unknown
7,2759,2SQFB,S,NaN,ENOG502SQFB,Function Unknown
9,2759,KOG0708,F,LarGe homolog,KOG0708,Nucleotide metabolism and transport
...,...,...,...,...,...,...
244186,2759,2SS10,S,TFIIB zinc-binding,ENOG502SS10,Function Unknown
244188,2759,2RWT5,S,NaN,ENOG502RWT5,Function Unknown
244190,2759,2TAAQ,S,NaN,ENOG502TAAQ,Function Unknown
244192,2759,2SU1H,S,NaN,ENOG502SU1H,Function Unknown


In [86]:
species_file = '../dollo_qfo/annotations/species_info_100223.csv'
spec_info = pd.read_csv(species_file)
spec_info.head(5)

,proteome_id,organism,organism_id,protein_count,taxon_mnemonic,tax_group,species_name
0,UP000008820,Aedes aegypti (Yellowfever mosquito) (Culex ae...,7159,20640,AEDAE,Eukaryota,Aedes aegypti
1,UP000008912,Ailuropoda melanoleuca (Giant panda),9646,38536,AILME,Eukaryota,Ailuropoda melanoleuca
2,UP000001631,Ajellomyces capsulatus (strain G186ARH82 / ATC...,447093,9214,AJECG,Eukaryota,Ajellomyces capsulatus
3,UP000007879,Amphimedon queenslandica (Sponge),400682,43437,AMPQE,Eukaryota,Amphimedon queenslandica
4,UP000001646,Anolis carolinensis (Green anole) (American ch...,28377,32752,ANOCA,Eukaryota,Anolis carolinensis


In [88]:
master_nog_file = '../dollo_qfo/data/allspecies_2759.mapping'
all_nogs = pd.read_csv(master_nog_file, sep='\t')
all_nogs = all_nogs[all_nogs.ProteinID!='ProteinID']
all_nogs[['up_ID','species_code']] = all_nogs['ProteinID'].str.split(pat='_', n=1, expand=True)
all_nogs['up_ID'] = all_nogs['ProteinID'].str.extract('(?<=\|)(.*)(?=\|)', expand=False)
all_nogs = all_nogs.merge(spec_info, how='left', left_on='species_code', right_on='taxon_mnemonic')
all_nogs

,ProteinID,ID,up_ID,species_code,proteome_id,organism,organism_id,protein_count,taxon_mnemonic,tax_group,species_name
0,tr|A0A023PGP0|A0A023PGP0_DANRE,KOG3863,A0A023PGP0,DANRE,UP000000437,Danio rerio (Zebrafish) (Brachydanio rerio),7955.0,46695.0,DANRE,Eukaryota,Danio rerio
1,tr|A0A024B5K5|A0A024B5K5_DANRE,KOG3624,A0A024B5K5,DANRE,UP000000437,Danio rerio (Zebrafish) (Brachydanio rerio),7955.0,46695.0,DANRE,Eukaryota,Danio rerio
2,tr|A0A068F9P7|A0A068F9P7_DANRE,KOG2246,A0A068F9P7,DANRE,UP000000437,Danio rerio (Zebrafish) (Brachydanio rerio),7955.0,46695.0,DANRE,Eukaryota,Danio rerio
3,tr|A0A097|A0A097_DANRE,ENOG502T0YK,A0A097,DANRE,UP000000437,Danio rerio (Zebrafish) (Brachydanio rerio),7955.0,46695.0,DANRE,Eukaryota,Danio rerio
4,tr|A0A097ZPG7|A0A097ZPG7_DANRE,KOG4161,A0A097ZPG7,DANRE,UP000000437,Danio rerio (Zebrafish) (Brachydanio rerio),7955.0,46695.0,DANRE,Eukaryota,Danio rerio
...,...,...,...,...,...,...,...,...,...,...,...
745630,tr|Q9ZWF5|Q9ZWF5_ORYSJ,KOG1715,Q9ZWF5,ORYSJ,UP000059680,Oryza sativa subsp. japonica (Rice),39947.0,48898.0,ORYSJ,Eukaryota,Oryza sativa subsp. japonica
745631,tr|Q9ZWR5|Q9ZWR5_ORYSJ,KOG1241,Q9ZWR5,ORYSJ,UP000059680,Oryza sativa subsp. japonica (Rice),39947.0,48898.0,ORYSJ,Eukaryota,Oryza sativa subsp. japonica
745632,tr|V9G2Z0|V9G2Z0_ORYSJ,KOG1940,V9G2Z0,ORYSJ,UP000059680,Oryza sativa subsp. japonica (Rice),39947.0,48898.0,ORYSJ,Eukaryota,Oryza sativa subsp. japonica
745633,sp|W0RYD3|SSG4_ORYSJ,ENOG502QTMX,W0RYD3,ORYSJ,UP000059680,Oryza sativa subsp. japonica (Rice),39947.0,48898.0,ORYSJ,Eukaryota,Oryza sativa subsp. japonica


In [104]:
human_nogs = all_nogs[all_nogs.species_code=='HUMAN']
human_leca = set(human_nogs[human_nogs.ID.isin(leca_nogs)]['ID'].to_list())

In [95]:
all_nogs[all_nogs.species_code.isna()]

,ProteinID,ID,up_ID,species_code,proteome_id,organism,organism_id,protein_count,taxon_mnemonic,tax_group,species_name


In [109]:
leca_annots = annots[annots['euNOG'].isin(leca_nogs)].reset_index(drop=True)
leca_annots['human_status'] = ['Present in humans' if i in human_leca else 'Lost in humans' for i in leca_annots['euNOG']]
leca_annots.to_csv('../dollo_qfo/annotations/leca_e5_annots.csv', index=False)